In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [3]:
# df = pd.read_csv("C:\\Users\\riffere\\OneDrive - Florida Panthers\\Documents\\LTP\\Kids_Club.csv")
# FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "kids_club_combined_data")

In [3]:
q = """
WITH initial as
    (select
        lower(parent_email) as parent_email,
        case
            when listagg(kc_group) like '%All-Star%' then 'All-Star'
            when listagg(kc_group) like '%Pro%' then 'Pro'
            else 'Rookie'
        end as kc_group_final
    from
        custom.kids_club_combined_data
    group by
        lower(parent_email)),
first_kids_club_season AS
    (select
        initial.parent_email, kc_group_final as kc_group, min(transaction_date) as transaction_date, min(season) as season
    from
        initial
    left join
        custom.kids_club_combined_data
        on initial.parent_email = kids_club_combined_data.parent_email
        and initial.kc_group_final = kids_club_combined_data.kc_group
    group by
        initial.parent_email, kc_group_final),
hist_acct_ids AS
    (SELECT
        acct_id, lower(email_addr) as email, date(add_date) as add_date
    FROM
        custom.arch_final_v_customer),
hist_emails_and_accounts AS
    (SELECT
         first_kids_club_season.parent_email, acct_id, CAST(MIN(season) AS VARCHAR) AS "season",
         CAST(MIN(transaction_date) AS date) AS "transaction_date", add_date
    FROM
        first_kids_club_season
    LEFT JOIN
        hist_acct_ids on first_kids_club_season.parent_email = hist_acct_ids.email
    WHERE
        first_kids_club_season.parent_email is not null and first_kids_club_season.parent_email <> ''
    GROUP BY
        first_kids_club_season.parent_email, acct_id, add_date),
historical_data AS
    (SELECT
        hist_emails_and_accounts.acct_id, parent_email, hist_emails_and_accounts.season, transaction_date, hist_emails_and_accounts.add_date,
        CASE
            WHEN cth_historical_all_1718_2223.season like '2017-18' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2017_18",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2018-19' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2018_19",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2019-20' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2019_20",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2020-21' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2020_21",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2021-22' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2021_22",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2022-23' and "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2022_23",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2017-18' then gross_revenue
            ELSE 0
        END AS "gross_rev_2017_18",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2018-19' then gross_revenue
            ELSE 0
        END AS "gross_rev_2018_19",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2019-20' then gross_revenue
            ELSE 0
        END AS "gross_rev_2019_20",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2020-21' then gross_revenue
            ELSE 0
        END AS "gross_rev_2020_21",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2021-22' then gross_revenue
            ELSE 0
        END AS "gross_rev_2021_22",
        CASE
            WHEN cth_historical_all_1718_2223.season like '2022-23' then gross_revenue
            ELSE 0
        END AS "gross_rev_2022_23"
    FROM
        hist_emails_and_accounts
    LEFT JOIN
        custom.cth_historical_all_1718_2223 on hist_emails_and_accounts.acct_id = cth_historical_all_1718_2223.acct_id),
historical_agg AS
    (SELECT
        parent_email, season, transaction_date, min(add_date) as add_date, max(is_stm_2017_18) as is_stm_2017_18,
        max(is_stm_2018_19) as is_stm_2018_19,
        max(is_stm_2019_20) as is_stm_2019_20, max(is_stm_2020_21) as is_stm_2020_21,
        max(is_stm_2021_22) as is_stm_2021_22, max(is_stm_2022_23) as is_stm_2022_23,
        sum(gross_rev_2017_18) as gross_rev_2017_18, sum(gross_rev_2018_19) as gross_rev_2018_19,
        sum(gross_rev_2019_20) as gross_rev_2019_20, sum(gross_rev_2020_21) as gross_rev_2020_21,
        sum(gross_rev_2021_22) as gross_rev_2021_22, sum(gross_rev_2022_23) as gross_rev_2022_23
    FROM
        historical_data
    GROUP BY
        parent_email, season, transaction_date),
current_acct_ids AS
    (SELECT
         crm_id as acct_id, lower(email) as email, date(creation_datetime) as add_date
    FROM
        custom.seatgeek_v_clients),
current_emails_and_accounts AS
    (SELECT
         first_kids_club_season.parent_email, acct_id, CAST(MIN(season) AS VARCHAR) AS "season",
         CAST(MIN(transaction_date) AS date) AS "transaction_date", add_date
    FROM
        first_kids_club_season
    LEFT JOIN
        current_acct_ids on first_kids_club_season.parent_email = current_acct_ids.email
    WHERE
        first_kids_club_season.parent_email is not null and first_kids_club_season.parent_email <> ''
    GROUP BY
        first_kids_club_season.parent_email, acct_id, add_date),
current_data AS
    (SELECT
         purch_client_crm_id as acct_id, parent_email, current_emails_and_accounts.season, current_emails_and_accounts.transaction_date,
         current_emails_and_accounts.add_date, gross_revenue as gross_rev_2023_24,
        CASE
            WHEN "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2023_24"
    FROM
        current_emails_and_accounts
    LEFT JOIN
        custom.cth_v_ticket_2324 on cth_v_ticket_2324.purch_client_crm_id = current_emails_and_accounts.acct_id),
current_agg AS
    (SELECT
        parent_email, season, transaction_date, min(add_date) as add_date,
        max(is_stm_2023_24) as is_stm_2023_24, sum(gross_rev_2023_24) as gross_rev_2023_24
    FROM
        current_data
    GROUP BY
        parent_email, season, transaction_date),
future_data AS
    (SELECT
         acct_id, parent_email, current_emails_and_accounts.season, current_emails_and_accounts.transaction_date,
         current_emails_and_accounts.add_date, cth_v_ticket_subscription_2425.gross_revenue as gross_rev_2024_25,
        CASE
            WHEN cth_v_ticket_subscription_2425.ticket_type IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2024_25"
    FROM
        current_emails_and_accounts
    LEFT JOIN
        custom.cth_v_ticket_subscription_2425 on cth_v_ticket_subscription_2425.purch_client_crm_id = current_emails_and_accounts.acct_id),
future_agg AS
    (SELECT
        parent_email, season, transaction_date, min(add_date) as add_date,
        max(is_stm_2024_25) as is_stm_2024_25, sum(gross_rev_2024_25) as gross_rev_2024_25
    FROM
        future_data
    GROUP BY
        parent_email, season, transaction_date)
SELECT
    historical_agg.parent_email, kc_group, historical_agg.season,
    historical_agg.transaction_date, coalesce(historical_agg.add_date, current_agg.add_date) as add_date,
    is_stm_2017_18, is_stm_2018_19,
    is_stm_2019_20, is_stm_2020_21, is_stm_2021_22, is_stm_2022_23, is_stm_2023_24, is_stm_2024_25,
    gross_rev_2017_18, gross_rev_2018_19, gross_rev_2019_20,
    gross_rev_2020_21, gross_rev_2021_22, gross_rev_2022_23, gross_rev_2023_24, gross_rev_2024_25
FROM
    historical_agg
LEFT JOIN
    current_agg on historical_agg.parent_email = current_agg.parent_email
LEFT JOIN
    future_agg on historical_agg.parent_email = future_agg.parent_email
LEFT JOIN
    first_kids_club_season on historical_agg.parent_email = first_kids_club_season.parent_email
ORDER BY
    parent_email
"""

In [4]:
df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [5]:
df['add_date'].fillna(datetime.strptime('2030-01-01', "%Y-%m-%d"), inplace = True)
df['add_date'] = pd.to_datetime(df['add_date']).dt.date

In [6]:
df['pre_kids_club_email'] = df.apply(lambda row: 0 if row['transaction_date'] == '' else (1 if row['add_date'] < row['transaction_date'] else 0), axis = 1)
df['post_kids_club_email'] = df.apply(lambda row: 0 if row['transaction_date'] == '' else (1 if row['add_date'] >= row['transaction_date'] else 0), axis = 1)

TypeError: '<' not supported between instances of 'datetime.date' and 'NoneType'

In [7]:
season_dict = {
    "2017-18" : 0,
    "2018-19" : 1,
    "2019-20" : 2,
    "2020-21" : 3,
    "2021-22" : 4,
    "2022-23" : 5,
    "2023-24" : 6
}

# sum season tickets before/after
for key, value in season_dict.items():
    col_list = list(df.columns[4:(4+value)])
    # print(col_list)
    df.loc[(df['season'] == key), 'pre_kids_club_ticktets'] =  df[col_list].sum(axis = 1)

for key, value in season_dict.items():
    col_list = list(df.columns[(4 + value):11])
    # print(col_list)
    df.loc[(df['season'] == key), 'post_kids_club_tickets'] =  df[col_list].sum(axis = 1)

# sum revenue before/after
for key, value in season_dict.items():
    col_list = list(df.columns[12:(12+value)])
    # print(col_list)
    df.loc[(df['season'] == key), 'pre_kids_club_revenue'] =  df[col_list].sum(axis = 1)


for key, value in season_dict.items():
    col_list = list(df.columns[(13 + value):20])
    # print(col_list)
    df.loc[(df['season'] == key), 'post_kids_club_revenue'] =  df[col_list].sum(axis = 1)

TypeError: unsupported operand type(s) for +: 'datetime.date' and 'int'

In [15]:
FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "kids_club_type_data")